#### Using the attached Reported Event and Aircraft Movement data, provide visualisations on the rate of events (per aircraft movement) at the four locations. Provide insight on any two event types and whether they involved civilian aircraft registered in Australia or overseas. Only present data for the 2022 calendar year (Jan-Dec).

Whilst we use a wide range of metrics to validate our performance, the following two internationally used
benchmark metrics are our key indicators of our safety performance:

• the required separation standard between aircraft or a restricted airspace volume is infringed (Loss
of Separations (LOS))

• an unauthorised aircraft, vehicle or person is on a runway (Runway Incursions). 

In [ ]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
data_dir = r"data"
events_path = os.path.join(data_dir,r"Reported_Events.csv")
blue_path = os.path.join(data_dir,r"Monthly_Aircraft_Movements_Blue_City.csv")
green_path = os.path.join(data_dir,r"Monthly_Aircraft_Movements_Green_City.csv")
red_path = os.path.join(data_dir,r"Monthly_Aircraft_Movements_Red_City.csv")
yellow_path = os.path.join(data_dir,r"Monthly_Aircraft_Movements_Yellow_City.csv")

In [ ]:
# Read in the city movement data, filter to 2022 only, store as one merged DF
blue_df = pd.read_csv(blue_path,names=['datetime','n_movements'],header=0)
blue_df['location'] = 'Blue City'
green_df = pd.read_csv(green_path,names=['datetime','n_movements'],header=0)
green_df['location'] = 'Green City'
red_df = pd.read_csv(red_path,names=['datetime','n_movements'],header=0)
red_df['location'] = 'Red City'
yellow_df = pd.read_csv(yellow_path,names=['datetime','n_movements'],header=0)
yellow_df['location'] = 'Yellow City'

movements_df = pd.concat([blue_df, green_df, red_df, yellow_df])
movements_df.datetime = pd.to_datetime(movements_df.datetime,format="%d/%m/%Y %H:%M:%S")
movements_df = movements_df[movements_df.datetime.dt.year==2022].copy().reset_index(drop=True)    
movements_df.sample(5)

In [ ]:
# Read in the events data, filter to 2022 only
events_df = pd.read_csv(events_path)
events_df.Event_Date = pd.to_datetime(events_df.Event_Date,format="%d-%m-%y")
events_df = events_df[events_df.Event_Date.dt.year==2022].copy().reset_index(drop=True)
events_df = events_df.rename(columns={'Location':'location'})
events_df.sample(5)

In [ ]:
# Get the counts for events and movements for each location, determine the rate per 1,000 movements
n_events = pd.DataFrame(events_df.location.value_counts()).rename(columns={'location':'n_events'})
n_movements = movements_df.groupby(['location']).sum()
results_df = pd.merge(n_events,n_movements,left_index=True,right_index=True)
results_df['event_rate']= results_df.n_events/(results_df.n_movements/1000)
results_df = results_df.reset_index().rename(columns={'index':'location'})
results_df = results_df.sort_values('location')
results_df

In [ ]:
# Define the bar colours based on city names
blue = (50, 116, 161)
green = (58, 146, 58)
red = (192, 61, 62)
yellow = (167, 168, 89)
colours = [(r/255, g/255, b/255, 1) for r, g, b in [blue,green,red,yellow]]

# Plot the results
ax = sns.barplot(data=results_df, x="location", y="event_rate", palette=colours)

# Add title, axis and tick labels
plt.title("Rate of Events – January to December 2022")
plt.ylabel("Event rate (per 1,000 movements)")
plt.xlabel("Location")
plt.xticks(rotation=45)

# Add annotations
for p in ax.patches:
    ax.annotate(format(p.get_height(), '.3f'), (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points', rotation=0)

# Add white space to top of plot
plt.ylim(0, ax.get_ylim()[1] * 1.3) 

plt.show()

#### Now that we've completed that, let's look into the monthly rates to see if there is anything interesting

In [ ]:
# Get a DataFrame with number of events by month and location
events_df['month'] = events_df.Event_Date.dt.to_period('M')
city_month_events_df = events_df.groupby(['location','month']).count().iloc[:,[0]].rename(columns={'Event_ID':'n_events'})
city_month_events_df = city_month_events_df.reset_index()
city_month_events_df.sample(5)

In [ ]:
# Merge the above with the movements dataframe joining on month and location, then calculate monthly event rates
movements_df['month'] = movements_df.datetime.dt.to_period('M')
merged_movements_events = pd.merge(movements_df,city_month_events_df,on=['month','location'])
merged_movements_events['event_rate'] = merged_movements_events.n_events/(merged_movements_events.n_movements/1000)
merged_movements_events = merged_movements_events.sort_values('location')
merged_movements_events.sample(5)

In [ ]:
# Plot each locations monthly event rates on one lineplot
sns.lineplot(x='datetime', y='event_rate', hue='location', data=merged_movements_events, palette=colours)
plt.show()

#### Looks like there are outliers in Red City for February and May 2022 - let's check that

In [ ]:
# Create a box plot with multiple categories
ax = sns.boxplot(x='location', y='event_rate', data=merged_movements_events,palette=colours)

sns.set(style="ticks")
plt.xlabel('Location')
plt.ylabel('Event Rate')
plt.title('Monthly Location Event Rate Box Plots')
plt.show()

#### Yes, they are outliers. Let's look into it further. 

In [ ]:
def get_combined_city_df(city: str, movements_df: pd.DataFrame, events_df: pd.DataFrame, blank_df: pd.DataFrame) -> pd.DataFrame:
    # Get the movements for this city
    city_movements = movements_df[movements_df.location==city][["month","n_movements"]]

    # Get the events for this city
    city_events_df = events_df[events_df.location==city]
    city_events_df = city_events_df.groupby(['Event_Type','month']).count().iloc[:,[0]].rename(columns={'Event_ID':'n_events'}).reset_index()
    
    # Join the city_events_df to blank df - this adds the 0 event types in a month to the data
    city_events_df = pd.merge(blank_df,city_events_df,on=['month','Event_Type'],how='left')
    city_events_df = city_events_df.fillna(0)
    
    # Add the movements values, calculate the event rates, sort by month and add the text month name
    city_events_df = pd.merge(city_events_df,city_movements,on="month")
    city_events_df['event_rate'] = city_events_df.n_events/(city_events_df.n_movements/1000)
    city_events_df = city_events_df.sort_values('month')
    city_events_df.month = city_events_df.month.dt.strftime('%B')
    
    return city_events_df

def show_event_rate_by_type_plots(df: pd.DataFrame, city: str) -> None:

    # Plot the event rates by month and type 
    fig = px.line(df, x="month", y="event_rate", color="Event_Type")
    fig.update_layout(xaxis_title='Month', yaxis_title="Event rate (per 1,000 movements)", legend_title="Event Type",title=f"2022 {city} Event Rates by Month and Type")
    fig.update_layout(yaxis_range=[0, 3])
    fig.show()
    
    # Plot the monthly event rate distributions as type box plots
    fig = px.box(df, 
                 x='Event_Type', 
                 y='event_rate', 
                 color='Event_Type')
    fig.update_layout(xaxis_title='Event Type', 
                      yaxis_title="Event rate (per 1,000 movements)", 
                      legend_title="Event Type",
                      title=f"2022 {city} Event Rate Distributions by Type")
    fig.update_layout(yaxis_range=[0, 3])
    fig.show()

In [ ]:
def get_blank_df(event_types: list, start: str, end: str) -> pd.DataFrame:
    # Create a temp DataFrame with rows for each month and event type combination
    months = pd.Series(pd.period_range(start=start, end=end, freq='M'))
    months = pd.concat([months] * len(event_types)).reset_index(drop=True)
    
    events = pd.Series(event_types)
    events = events.repeat(len(months)/len(event_types)).reset_index(drop=True)
    
    return pd.DataFrame({'month':months,'Event_Type':events})

In [ ]:
blank_df = get_blank_df(events_df.Event_Type.unique(),'2022-01','2022-12')
city = "Red City"
show_event_rate_by_type_plots(get_combined_city_df(city,movements_df,events_df,blank_df),city)

#### Now that we look closer, there is an outlier for the Facility Issue event type in May, but no event level outliers in Feb. Let's see what type of aircraft were involved in the facility issues at Red City for 2022.

In [ ]:
events_df[(events_df.location=="Red City")&(events_df.Event_Type=="Facility Issue")].Aircraft_Register.value_counts()

We see that of the civilian aircraft involved in Facility Issues at Red City in 2022, all 14 of them were Australian registered.

#### Let's look for any patterns of interest in the other cities now

In [ ]:
city = "Blue City"
show_event_rate_by_type_plots(get_combined_city_df(city,movements_df,events_df,blank_df),city)

In [ ]:
city = "Yellow City"
show_event_rate_by_type_plots(get_combined_city_df(city,movements_df,events_df,blank_df),city)

In [ ]:
city = "Green City"
show_event_rate_by_type_plots(get_combined_city_df(city,movements_df,events_df,blank_df),city)

#### Loss of separation (LOS) and runway incursions are of interest given the risk of loss of life involved. These are also internationally used benchmark metrics and key indicators of safety performance. 

#### We can see that there is an outlier for LOS in February for Green City. Let's look into that. 

In [ ]:
events_df[(events_df.location=="Green City")&(events_df.Event_Type=="Loss of Separation")]


We see that of the civilian aircraft involved in LOS at Green City in 2022, 3 of them were Australian registered and 2 not recorded. It's worth noting that all of them were listed with a date of 25 February

In [ ]:
events_df[(events_df.location=="Red City")&(events_df.Event_Type=="Facility Issue")&(events_df.month=="2022-05")]

In [ ]:
events_df.Event_Date.value_counts().sort_values()

In [ ]:
events_df[events_df.Event_Date=="2022-03-08"]

In [ ]:
events_df[(events_df.location=="Green City")&(events_df.Event_Type=="Go Around")&(events_df.month=="2022-12")]